# Natural Language Processing

## 필수과제 3: Subword-level Language Model with LSTM

> Reference: https://github.com/pytorch/examples/tree/master/word_language_model




In [1]:
from google.colab import drive
drive.mount('/content/drive')b

Mounted at /content/drive


### 0. 데이터 업로드


1. Boostcourse [필수 과제] Subword-level Language Model 에서 `wikitext-2.zip` 파일을 다운받습니다.
2. 본 Colab 환경에 `train.txt`, `dev.txt`, `test.txt` 파일을 업로드합니다.
3. `%ls` command 를 실행했을 때, `sample_data/  test.txt  train.txt  valid.txt` 가 나오면 성공적으로 데이터 준비가 완료된 것 입니다.


In [19]:
from google.colab import files
file_uploaded = files.upload()

In [21]:
%ls

drive/  sample_data/  test.txt  train.txt  valid.txt


In [12]:
# EDA(Exploratory Data Analysis)
path_train = './train.txt'
with open(path_train, 'r', encoding="utf8") as f:
    corpus_train = f.readlines()    

# train dataset 크기 확인
print(len(corpus_train))

# 처음 10 문장을 print 해 봅시다.
for sent in corpus_train[:10]:
    print(sent)

36718
 

 = Valkyria Chronicles III = 

 

 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcom

In [4]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 895 kB 43.2 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 636 kB 52.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Introduction


* 본 과제의 목적은 Subword tokenization의 필요성을 직접 느껴보는 것입니다.
* Subword tokenization 기반 language model을 구현하면서 이전 과제의 Word-level language model과 비교해보는 시간을 갖겠습니다. 추가적으로 RNN을 LSTM으로 변경했을 때의 성능 차이에 대해 살펴보겠습니다.
*   Subword-level language model을 구현하고, 주어진 데이터를 가공하여 모델을 학습한 후 학습된 언어 모델을 이용해 문장을 생성합니다.
*   **ANSWER HERE** 이라고 작성된 부분을 채워 완성하시면 됩니다. 다른 부분의 코드를 변경하면 오류가 발생할 수 있습니다.



```
💡 Subword는 무엇인가요?

Subword는 하나의 단어를 여러개의 단위로 분리했을 때 하나의 단위를 나타냅니다. "subword"를 subword 단위로 나타낸 하나의 예시는 다음과 같습니다.

"sub" + "word"

sub라는 접두사와 word라고 하는 어근으로 나누어 "subword"라고 하는 word를 2개의 subword로 나타냈습니다.

이외에도 다양한 형태의 subword로 나타낼 수 있습니다. (e.g., "su" + "bword", "s" + "ubword", "subwor" + "d")
```

```
💡 tokenization은 무엇인가요?

tokenization은 주어진 입력 데이터를 자연어처리 모델이 인식할 수 있는 단위로 변환해주는 방법입니다. 

💡 word tokenization은요?

word tokenization의 경우 "단어"가 자연어처리 모델이 인식하는 단위가 됩니다.
"I have a meal"이라고 하는 문장을 가지고 word tokenization을 하면 다음과 같습니다. 

- ['I', 'have', 'a', 'meal']

영어의 경우 대부분 space를 기준으로 단어가 정의되기 때문에 .split()을 이용해 쉽게 word tokenization을 구현할 수 있습니다.
영어에서 word tokenization은 space tokenization이라고도 할 수 있고, 
subword tokenization 이전에 수행되는 pre-tokenization 방법으로도 많이 사용됩니다.

"나는 밥을 먹는다"라는 문장을 word tokenization하면 다음과 같습니다.
- ['나', '는', '밥', '을', '먹는다']

한국어에서 "단어"는 공백(space)을 기준으로 정의되지 않습니다. 이는 한국어가 갖고 있는 "교착어"로서의 특징 때문입니다. 
체언 뒤에 조사가 붙는 것이 대표적인 특징이며 의미 단위가 구분되고 자립성이 있기 때문에 조사는 "단어"입니다.

한국어에서는 pre-tokenization 방법으로 space tokenization을 사용하지 않고 형태소 분석기를 활용하고 있습니다.
```

(참고1: [국립 국어원: "조사는 단어이다"](https://www.korean.go.kr/front/onlineQna/onlineQnaView.do?mn_id=216&qna_seq=26915#:~:text='%EC%A1%B0%EC%82%AC'%EB%8A%94%20%EC%99%84%EC%A0%84%ED%95%9C%20%EC%9E%90%EB%A6%BD%EC%84%B1%EC%9D%80,%ED%95%98%EC%97%AC%20%EB%8B%A8%EC%96%B4%EB%A1%9C%20%EC%B2%98%EB%A6%AC%ED%95%A9%EB%8B%88%EB%8B%A4.) )

(참고2: [Huggingface: Pre-tokenization](https://huggingface.co/docs/tokenizers/python/latest/pipeline.html#pre-tokenization))

(참고3: [Konlpy: 형태소 분석기](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/))

```
💡 그럼 Subword tokenization은 무엇인가요?

Subword tokenizaiton은 말 그대로 subword 단위로 tokenization을 한다는 뜻입니다.
방금 전 word tokenization을 수행했던 문장을 이용해 subword tokenization을 수행한 예시를 보겠습니다.

Subword tokenization을 적용했을 때는 다음과 같이 tokenization이 될 수 있습니다.

Example 1

"I have a meal" -> ['I', 'hav', 'e', 'a', 'me', 'al']
"나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹는', '다']

word 단위가 아니라 그보다 더 잘게 쪼갠 subword 단위로 문장을 tokenization합니다.

위에서 말씀드린 것과 같이 여러가지 경우의 수가 가능합니다.

Example 2

"I have a meal" -> ['I', 'ha', 've', 'a', 'mea', 'l']
"나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹', '는다']

그렇지만 기본적으로 공백을 넘어선 subword를 구성하진 않습니다.
예를 들어 다음과 같이 tokenizaiton을 수행하진 않습니다.

Example 3

"I have a meal" -> ['Iha', 've', 'am', 'ea', 'l']
"나는 밥을 먹는다" -> ['나는밥', '을먹', '는다']
```

(참고4: [Huggingface: subword-tokenization](https://huggingface.co/transformers/tokenizer_summary.html#subword-tokenization))

```
💡 Subword tokenization은 왜 필요한가요?

이전 과제에서 사용했던 word tokenization 코드를 불러와 그 필요성을 생각해 봅시다.

```

In [13]:
import os
from io import open
import torch

In [14]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {'<unk>': 0}
        self.idx2word = ['<unk>']

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

In [15]:
class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(os.path.join(path, 'train.txt'), 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    try:
                        ids.append(self.dictionary.word2idx[word])
                    except:
                        print(word)
                        ids.append(0)
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

In [16]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, input, hidden):
        ############################ ANSWER HERE ################################
        # TODO: forward 함수를 완성해주세요.
        #
        # Hint 1: 필수 과제 2의 forward 함수와 정답이 동일합니다.
        #########################################################################

        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

In [17]:
import time
import math
import os
import torch
import torch.nn as nn

import easydict
args = easydict.EasyDict({
    "data"    : './data/wikitext-2',    # location of the data corpus
    "model"   : 'RNN_TANH',             # type of recurrent net (RNN_TANH, RNN_RELU, LSTM, GRU)
    "emsize"  : 200,                    # size of word embeddings
    "nhid"    : 512,                    # number of hidden units per layer
    "nlayers" : 2,                      # number of layers
    "lr"      : 20,                     # initial learning rate
    "clip"    : 0.25,                   # gradient clipping
    "epochs"  : 6,                      # upper epoch limit
    "batch_size": 20,                   # batch size
    "bptt"    : 35,                     # sequence length
    "dropout" : 0.2,                    # dropout applied to layers (0 = no dropout)
    "seed"    : 1111,                   # random seed
    "cuda"    : True,                   # use CUDA
    "log_interval": 200,                # report interval
    "save"    : 'model.pt',             # path to save the final model
    "dry_run" : True,                   # verify the code and the model

})

# 디바이스 설정
device = torch.device("cuda" if args.cuda else "cpu")

```
train.txt의 문장들을 word tokenization 해보고 단어들의 개수를 세어보겠습니다
```

In [26]:
corpus = Corpus('./')
ntokens = len(corpus.dictionary)
print(ntokens)

33278


```
이전 과제에 사용된 embedding dimension의 크기는 200이므로 word embedding에 사용된 parameter의 수는 33278 x 200 (6,655,600개)입니다.

그렇다면, RNN 모델에 사용되는 weight의 parameter 개수는 몇개인지 간단한 함수를 이용해 확인해보겠습니다
```

In [27]:
model = RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout)

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [29]:
print(f"Word embedding parameter 개수: {count_parameters(model.encoder)}")
print(f"RNN parameter 개수: {count_parameters(model.rnn)}")

Word embedding parameter 개수: 6655600
RNN parameter 개수: 890880


```
💡 RNN parameter, Word embedding parameter 개수를 비교해보면 word embedding parameter의 개수가 RNN 모델의 parameter보다 압도적으로 많습니다.

word embedding을 사용하는 경우 training에 사용되는 text file의 크기가 커질수록 word embedding parameter는 
더 커지게 되고 전체 parameter 대비 word embedding이 차지하는 비중은 매우 높아집니다.
```

```
✨ 이런 parameter 비중의 비대칭성을 해결하기 위해 처음에는 character-level tokenization 방법이 주목을 받았습니다. 
말 그대로 하나의 글자를 기준으로 tokenization을 하는건데요.
이전 예시를 character 기반 tokenization을 하면 다음과 같습니다.

"I have a meal" -> ['I', 'h', 'a', 'v', 'e', 'a', 'm', 'e', 'a', 'l']
"나는 밥을 먹는다" -> ['나', '는', '밥', '을', '먹', '는', '다']

그러나, character 기반 tokenization 역시 지나치게 긴 sequence 길이, 성능 저하 등의 문제를 겪으며 
subword tokenization이 각광을 받게 되었습니다.

❓ word tokenization 기법과 구분되는 subword tokenization의 장점을 하나만 더 생각해볼까요?

subword tokenization의 장점은 Out-of-vocabulary (OOV) 문제에서 상대적으로 자유롭다는 것입니다.

일반적으로 subword들은 최소 철자 단위에서 하나씩 더 긴 subword를 추가하는 방식으로 만들어집니다.

예를 들어, 영어의 경우 a~z의 알파벳부터 시작해서 두글자, 세글자, 네글자 subword 등으로 확장해나가며 
subword를 추가해 단어를 구성하고 이를 바탕으로 subword tokenization을 수행하기 때문에 다른 언어를 tokenization하지 않는다면 
OOV 문제에서 자유롭다고 볼 수 있습니다.

대표적인 subword tokenization에 사용되는 algorithm 중 하나인 byte pair encoding은 선택 과제 3에서 살펴볼 수 있으니 참고해주세요 !

✨ 그럼 이제부터 BERT 모델에서 사용한 subword tokenization algorithm을 이용해 language modeling task를 수행해보겠습니다. 
subword tokenizer는 transformers 라이브러리를 이용해 쉽게 불러올 수 있습니다.
```

(참고5: [Huggingface: subword tokenization](https://huggingface.co/transformers/tokenizer_summary.html#subword-tokenization))


In [30]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
# subword tokenization 예시
print(tokenizer.tokenize('Making AI Beneficial'))
print(tokenizer.tokenize('Boostcamp AI Tech'))

['Making', 'AI', 'Ben', '##ef', '##ici', '##al']
['Bo', '##ost', '##cam', '##p', 'AI', 'Tech']


In [32]:
class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        ############################ ANSWER HERE ################################
        # TODO: tokenizer 객체를 활용해 tokenize 함수를 완성해주세요.
        #
        # Hint1: 이번 과제에서는 각 데이터셋의 문장들을 subword로 tokenization하여
        #        Dictionary에 word로 담아주세요.
        #        
        #########################################################################
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = tokenizer.tokenize(line.strip()) + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = tokenizer.tokenize(line.strip()) + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

```
그러면 이제 다시 모델을 선언하고 parameter의 개수를 살펴보겠습니다.
```

In [34]:
subword_corpus = Corpus('./')
ntokens = len(subword_corpus.dictionary)
subwordmodel = RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout)

In [35]:
print(f"Word embedding parameter 개수: {count_parameters(subwordmodel.encoder)}")
print(f"RNN parameter 개수: {count_parameters(subwordmodel.rnn)}")

Word embedding parameter 개수: 4619000
RNN parameter 개수: 890880


```
이전에 비해 embedding parameter 개수는 확연히 줄어들었습니다.
6,655,600개 -> 4,619,000개

그러면 이제 subword 기반의 언어 모델 성능을 살펴보겠습니다.
```

In [36]:
###############################################################################
# Load data
###############################################################################

# Starting from sequential data, batchify arranges the dataset into columns.
# For instance, with the alphabet as the sequence and batch size 4, we'd get
# ┌ a g m s ┐
# │ b h n t │
# │ c i o u │
# │ d j p v │
# │ e k q w │
# └ f l r x ┘.
# These columns are treated as independent by the model, which means that the
# dependence of e. g. 'g' on 'f' can not be learned, but allows more efficient
# batch processing.

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(subword_corpus.train, args.batch_size)
val_data = batchify(subword_corpus.valid, eval_batch_size)
test_data = batchify(subword_corpus.test, eval_batch_size)

In [37]:
###############################################################################
# Build the model
###############################################################################

ntokens = len(subword_corpus.dictionary)

model = RNNModel(args.model, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout).to(device)

criterion = nn.NLLLoss()

In [38]:
###############################################################################
# Training code1 - define functions
###############################################################################

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)


# get_batch subdivides the source data into chunks of length args.bptt.
# If source is equal to the example output of the batchify function, with
# a bptt-limit of 2, we'd get the following two Variables for i = 0:
# ┌ a g m s ┐ ┌ b h n t ┐
# └ b h n t ┘ └ c i o u ┘
# Note that despite the name of the function, the subdivison of data is not
# done along the batch dimension (i.e. dimension 1), since that was handled
# by the batchify function. The chunks are along dimension 0, corresponding
# to the seq_len dimension in the LSTM.

def get_batch(source, i):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(subword_corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args.bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(subword_corpus.dictionary)
    hidden = model.init_hidden(args.batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()

        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden)

        ############################ ANSWER HERE ################################
        # TODO: train 함수를 (이곳에) 완성해주세요.
        #
        # Hint1: 필수 과제 2의 정답과 동일합니다.
        #########################################################################
        loss = criterion(output, targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        total_loss += loss.item()

        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        if args.dry_run:
            break

In [39]:
###############################################################################
# Training code2 - run 
###############################################################################

# Loop over epochs.
lr = args.lr
best_val_loss = None

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, args.epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open(args.save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f)
    # after load the rnn params are not a continuous chunk of memory
    # this makes them a continuous chunk, and will speed up forward pass
    # Currently, only rnn model supports flatten_parameters function.
    if args.model in ['RNN_TANH', 'RNN_RELU', 'LSTM', 'GRU']:
        model.rnn.flatten_parameters()

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  8.70s | valid loss  8.75 | valid ppl  6332.34
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  8.42s | valid loss 12.95 | valid ppl 422869.79
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  8.48s | valid loss 11.59 | valid ppl 107958.89
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  8.48s | valid loss 10.16 | valid ppl 25798.69
------------------------------------------------------------------------

### 4. 학습한 언어 모델로 문장 생성


*   학습이 완료된 모델을 불러와 random 한 단어를 input 으로 넣어준 후 정해진 개수의 단어를 생성합니다.
*   생성한 문장을 decode 하여 (즉, idx2word 를 이용해 id 를 word 로 변환하여) generate.txt 파일에 저장합니다. 

generate.txt 파일을 열어 언어모델 생성결과를 확인해보세요.



In [41]:
###############################################################################
# Language Modeling on Wikitext-2
#
# This file generates new sentences sampled from the language model
#
###############################################################################

import torch

# Model parameters.
test_args = easydict.EasyDict({
    "data"      : './',  # location of data corpus
    "checkpoint": './model.pt',         # model checkpoint to use
    "outf"      : 'generate.txt',       # output file for generated text
    "words"     : 1000,                 # number of words to generate
    "seed"      : 1111,                 # random seed
    "cuda"      : True,                 # use CUDA
    "temperature": 1.0,                 # temperature - higher will increase diversity
    "log_interval": 100                 # reporting interval
})

# Set the random seed manually for reproducibility.
torch.manual_seed(test_args.seed)
if torch.cuda.is_available():
    if not test_args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda" if test_args.cuda else "cpu")

if test_args.temperature < 1e-3:
    parser.error("--temperature has to be greater or equal 1e-3")

with open(test_args.checkpoint, 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

corpus = Corpus(test_args.data)
ntokens = len(subword_corpus.dictionary)

hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with open(test_args.outf, 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(test_args.words):
            output, hidden = model(input, hidden)
            word_weights = output.squeeze().div(test_args.temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]

            outf.write(word + ('\n' if i % 20 == 19 else ' '))

            if i % test_args.log_interval == 0:
                print('| Generated {}/{} words'.format(i, test_args.words))

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words
